In [1]:
import os

from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/standalone/.env")

import import_ipynb
from utils import to_timestamp, df_data_sensors, to_be_checked, is_corr_ok, df_to_csv, df_from_csv

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from aqi_epa_pm25.ipynb
PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150


## PM2.5 Raw Data

In [2]:
# Load Tangaras
df_tangaras = df_from_csv("tangaras.csv", dtindex=False)

print(f"Total Tangara Sensors: {len(df_tangaras)}")

df_tangaras.head()

Total Tangara Sensors: 12


,ID,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
0,TANGARA_25CE,d29e4r0,D29ESP32DE125CE,3.3789825439453125 -76.54106140136719,3.378983,-76.541061
1,TANGARA_260A,d29edyj,D29ESP32DE1260A,3.4613800048828125 -76.51222229003906,3.461380,-76.512222
2,TANGARA_4B1A,d29esj8,D29ESP32DE94B1A,3.4586334228515625 -76.46415710449219,3.458633,-76.464157
3,TANGARA_14D6,d29eg4k,D29ESP32DED14D6,3.4847259521484375 -76.50260925292969,3.484726,-76.502609
4,TANGARA_1EE6,d29edh3,D29ESP32DED1EE6,3.4517669677734375 -76.55067443847656,3.451767,-76.550674


In [3]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
START_ISO8601_DATETIME=os.getenv("START_ISO8601_DATETIME", None)
start_timestamp = to_timestamp(START_ISO8601_DATETIME)
# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
END_ISO8601_DATETIME=os.getenv("END_ISO8601_DATETIME", None)
end_timestamp = to_timestamp(os.getenv("END_ISO8601_DATETIME", None))

print(f'Since: {START_ISO8601_DATETIME} -> {start_timestamp}, Until: {END_ISO8601_DATETIME} -> {end_timestamp}')

Since: 2023-09-26T00:00:00-05:00 -> 1695704400000, Until: 2023-09-26T00:05:00-05:00 -> 1695704700000


In [4]:
# PM2.5 Data Frame Sensors
df_pm25_raw = df_data_sensors(df_tangaras, start_timestamp, end_timestamp, 'pm25')
df_pm25_raw.head()
#df_pm25_raw.shape

,TANGARA_25CE,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_1EE6,TANGARA_2E9A,TANGARA_2FF6,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
DATETIME,,,,,,,,,,,,
2023-09-26 00:00:30-05:00,1.0,6.0,5.0,7.0,2.0,11.0,7.0,14.0,3.0,7.0,5.0,0.0
2023-09-26 00:01:00-05:00,1.0,6.0,7.0,6.0,3.0,12.0,8.0,13.0,7.0,9.0,5.0,0.0
2023-09-26 00:01:30-05:00,1.0,7.0,8.0,6.0,4.0,11.0,8.0,12.0,7.0,NaN,5.0,0.0
2023-09-26 00:02:00-05:00,0.0,7.0,8.0,6.0,4.0,11.0,8.0,14.0,7.0,11.0,5.0,0.0
2023-09-26 00:02:30-05:00,1.0,6.0,8.0,6.0,3.0,9.0,9.0,13.0,5.0,10.0,5.0,0.0


## Descriptive Statistics

In [5]:
# Describe Data
df_pm25_raw.describe()

,TANGARA_25CE,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_1EE6,TANGARA_2E9A,TANGARA_2FF6,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,8.000000,9.000000,9.0
mean,0.888889,6.333333,7.333333,5.777778,3.000000,10.666667,8.777778,13.222222,5.777778,9.375000,4.888889,0.0
std,0.333333,0.500000,1.000000,0.666667,0.707107,1.000000,1.092906,0.666667,1.394433,1.187735,0.333333,0.0
min,0.000000,6.000000,5.000000,5.000000,2.000000,9.000000,7.000000,12.000000,3.000000,7.000000,4.000000,0.0
25%,1.000000,6.000000,7.000000,5.000000,3.000000,10.000000,8.000000,13.000000,5.000000,9.000000,5.000000,0.0
50%,1.000000,6.000000,8.000000,6.000000,3.000000,11.000000,9.000000,13.000000,6.000000,9.500000,5.000000,0.0
75%,1.000000,7.000000,8.000000,6.000000,3.000000,11.000000,10.000000,14.000000,7.000000,10.000000,5.000000,0.0
max,1.000000,7.000000,8.000000,7.000000,4.000000,12.000000,10.000000,14.000000,7.000000,11.000000,5.000000,0.0


## Missing Data

In [6]:
# Missing Data Threshold
threshold = 90
print(f'Threshold: {threshold}%')

# For each Tangara sensor
for id_tangara_sensor in df_pm25_raw.columns:
    df_pm25_sensor = df_pm25_raw[[id_tangara_sensor]]
    # To be checked
    is_ok, data_percent, missing_data_percent = to_be_checked(df_pm25_sensor, threshold)
    if not is_ok:
        print(f"Tangara Sensor: {id_tangara_sensor}, Data: {data_percent}%, Missing: {missing_data_percent}%, To be checked")

Threshold: 90%
Tangara Sensor: TANGARA_532E, Data: 89%, Missing: 11%, To be checked


## Data Correlation

In [7]:
# Data Correlation Threshold
threshold = 0.9
print(f'Threshold: {threshold}%')

ID_REFE_TANGARA_SENSOR='TANGARA_06BE'
ID_TARG_TANGARA_SENSOR='TANGARA_2FF6'
# Reference Tangara Sensor
df_reference_sensor = df_pm25_raw[ID_REFE_TANGARA_SENSOR]
# Target Tangara Sensor
df_target_sensor = df_pm25_raw[ID_TARG_TANGARA_SENSOR]

# To be checked
is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
if not is_ok:
    print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {ID_TARG_TANGARA_SENSOR}, Correlation: {corr}, To be checked")

# For each Tangara sensor
# for id_tangara_sensor in df_pm25_raw.columns:
#     # Target Tangara Sensor
#     df_target_sensor = df_pm25_raw[id_tangara_sensor]
#     # To be checked
#     is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
#     if not is_ok:
#         print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {id_tangara_sensor}, Correlation: {corr}, To be checked")


Threshold: 0.9%
Reference Tangara Sensor: TANGARA_06BE, Target Tangara Sensor: TANGARA_2FF6, Correlation: 0.0, To be checked


In [8]:
# Save PM2.5 Data Frame Sensors into CSV file
df_to_csv(df_pm25_raw, "pm25_raw.csv")